In [26]:
import os
os.environ['PATH']
stream = os.popen('owltools --help')
output = stream.read()
output

"owltools [ONTOLOGY ...] [COMMAND ...]\n\nCommands/Options\n  (type 'owltools COMMAND -h' for more info)\n    --pellet\n    --hermit\n    --elk\n    --jfact\n    --more\n    --use-reasoner|--set-reasoner-name\n    --no-dispose\n    --reasoner\n    --init-reasoner\n    --reasoner-dispose\n    --reasoner-flush\n    --no-reasoner\n    --log-info\n    --log-warning\n    --log-debug\n    --log-error\n    --no-debug\n    --no-logging\n    --silence-elk\n    --monitor-memory\n    --list-classes\n    --object-to-label-table\n    --write-all-subclass-relationships\n    --query-ontology\n    --merge\n    --use-catalog\n    --use-catalog-xml\n    --catalog-xml\n    --map-ontology-iri\n    --auto-ontology-iri\n    --remove-imports-declarations\n    --remove-import-declaration\n    --add-imports-declarations\n    --subtract\n    --diff\n    --set-ontology-id\n    --add-ontology-annotation\n    --create-ontology\n    --merge-import\n    --merge-import-closure\n    --merge-imports-closure\n    --merg

In [3]:
from rdflib import Graph
import pandas as pd
# use ontobio!!

# Analysing ROBOT Report

g = Graph()
g.parse(mondo_file, format="xml")

from ontobio.ontol_factory import OntologyFactory
mondo_file="/Users/matentzn/ws/mondo/src/ontology/mondo-qc.obo"
ofa = OntologyFactory()
ont = ofa.create(mondo_file)
ont.get_roots()[:3]

In [13]:
report = '/Users/matentzn/ws/mondo/src/ontology/reports/obo-report.tsv'
df = pd.read_csv(report,sep='\t')
# Bug in ROBOT, wrong number of columns:
df.columns=['Rule Name', 'Subject', 'Property', 'Value','Trash']
df['Level'] = df.index
df.reset_index(level=0, inplace=True)
df

,index,Rule Name,Subject,Property,Value,Trash,Level
0,ERROR,duplicate_definition,HP:0032154,IAO:0000115,Oral aphthous ulcers typically present as pain...,NaN,ERROR
1,ERROR,duplicate_definition,MONDO:0005318,IAO:0000115,Oral aphthous ulcers typically present as pain...,NaN,ERROR
2,ERROR,duplicate_definition,HP:0000491,IAO:0000115,Inflammation of the cornea.,NaN,ERROR
3,ERROR,duplicate_definition,MONDO:0003085,IAO:0000115,Inflammation of the cornea.,NaN,ERROR
4,ERROR,duplicate_definition,CP:0000000,IAO:0000115,Cytoplasm that exhibits molecular interaction ...,NaN,ERROR
...,...,...,...,...,...,...,...
18403,INFO,lowercase_definition,http://purl.obolibrary.org/obo/uberon/core#tra...,IAO:0000115,.,NaN,INFO
18404,INFO,lowercase_definition,http://purl.obolibrary.org/obo/uberon/core#tra...,IAO:0000115,.,NaN,INFO
18405,INFO,lowercase_definition,http://purl.obolibrary.org/obo/uberon/core#tra...,IAO:0000115,.,NaN,INFO
18406,INFO,lowercase_definition,https://www.wikidata.org/wiki/Q2,IAO:0000115,third planet from the Sun in the Solar System,NaN,INFO


In [15]:
df=df[df['Subject'].str.startswith('MONDO')]
df[['Level','Rule Name','Value']].groupby(['Level','Rule Name']).count()

Value
Level Rule Name                         
ERROR deprecated_class_reference       7
      duplicate_definition             8
      duplicate_label                  3
      multiple_equivalent_classes      4
INFO  lowercase_definition           383
WARN  duplicate_exact_synonym       1689
      duplicate_scoped_synonym      1597
      missing_definition               0

In [27]:
df[df['Rule Name']=="lowercase_definition"]
# Revealed a more complex issue we need to sort first: https://github.com/monarch-initiative/mondo/issues/1979

,index,Rule Name,Subject,Property,Value,Trash,Level
17595,INFO,lowercase_definition,MONDO:0000238,IAO:0000115,a mild form of bubonic plague characterized by...,NaN,INFO
17596,INFO,lowercase_definition,MONDO:0000241,IAO:0000115,a congestive cardiomyopathy caused by a combin...,NaN,INFO
17597,INFO,lowercase_definition,MONDO:0000455,IAO:0000115,an inherited ocular disorder characterized by ...,NaN,INFO
17598,INFO,lowercase_definition,MONDO:0000712,IAO:0000115,frontotemporal dementia plus amyotrophic later...,NaN,INFO
17599,INFO,lowercase_definition,MONDO:0001195,IAO:0000115,a type of tick-borne disease which presents on...,NaN,INFO
...,...,...,...,...,...,...,...
17973,INFO,lowercase_definition,MONDO:0043314,IAO:0000115,a skin condition caused by Mycobacterium marin...,NaN,INFO
17974,INFO,lowercase_definition,MONDO:0043549,IAO:0000115,a medical condition characterized by major sho...,NaN,INFO
17975,INFO,lowercase_definition,MONDO:0044255,IAO:0000115,<Subhead> Genetic Heterogeneity of Variation i...,NaN,INFO
17976,INFO,lowercase_definition,MONDO:0044266,IAO:0000115,"{2,3:Berg and Bearn (1966, 1966)} discovered a...",NaN,INFO


In [17]:
df[df['Rule Name']=="deprecated_class_reference"]
# https://github.com/monarch-initiative/mondo/issues/1982

,index,Rule Name,Subject,Property,Value,Trash,Level
248,ERROR,deprecated_class_reference,MONDO:0001328,owl:someValuesFrom,GO:0004887,NaN,ERROR
249,ERROR,deprecated_class_reference,MONDO:0010142,owl:someValuesFrom,GO:0004887,NaN,ERROR
250,ERROR,deprecated_class_reference,MONDO:0010186,owl:someValuesFrom,GO:0008434,NaN,ERROR
251,ERROR,deprecated_class_reference,MONDO:0010966,owl:someValuesFrom,MONDO:0000746,NaN,ERROR
252,ERROR,deprecated_class_reference,MONDO:0019154,owl:someValuesFrom,GO:0004882,NaN,ERROR
253,ERROR,deprecated_class_reference,MONDO:0020186,rdfs:subClassOf,MONDO:0016033,NaN,ERROR
254,ERROR,deprecated_class_reference,MONDO:0024239,owl:someValuesFrom,GO:0072358,NaN,ERROR


In [18]:
df[df['Rule Name']=="duplicate_definition"]
# https://github.com/monarch-initiative/mondo/issues/1975

,index,Rule Name,Subject,Property,Value,Trash,Level
1,ERROR,duplicate_definition,MONDO:0005318,IAO:0000115,Oral aphthous ulcers typically present as pain...,NaN,ERROR
3,ERROR,duplicate_definition,MONDO:0003085,IAO:0000115,Inflammation of the cornea.,NaN,ERROR
6,ERROR,duplicate_definition,MONDO:0012676,IAO:0000115,Any autosomal recessive malignant osteopetrosi...,NaN,ERROR
7,ERROR,duplicate_definition,MONDO:0012679,IAO:0000115,Any autosomal recessive malignant osteopetrosi...,NaN,ERROR
9,ERROR,duplicate_definition,MONDO:0001014,IAO:0000115,A slowly progressing leukemia characterized by...,NaN,ERROR
10,ERROR,duplicate_definition,MONDO:0007303,IAO:0000115,A rib that is attached to a cervical vertebra.,NaN,ERROR
12,ERROR,duplicate_definition,MONDO:0001868,IAO:0000115,A form of glaucoma in which there is no visibl...,NaN,ERROR
13,ERROR,duplicate_definition,MONDO:0007665,IAO:0000115,A form of glaucoma in which there is no visibl...,NaN,ERROR


In [19]:
df[df['Rule Name']=="duplicate_label"]
# https://github.com/monarch-initiative/mondo/issues/1972

,index,Rule Name,Subject,Property,Value,Trash,Level
260,ERROR,duplicate_label,MONDO:0007955,rdfs:label,Meckel's diverticulum,NaN,ERROR
262,ERROR,duplicate_label,MONDO:0045042,rdfs:label,localized,NaN,ERROR
268,ERROR,duplicate_label,MONDO:0007303,rdfs:label,cervical rib,NaN,ERROR


In [20]:
df[df['Rule Name']=="multiple_equivalent_classes"]
# https://github.com/ontodev/robot/issues/733
# Ignore 0021108 0045024 until ticket resolved.

,index,Rule Name,Subject,Property,Value,Trash,Level
34,ERROR,multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,a77fd83f-bc94-4da8-9049-2f241727b854genid324061,NaN,ERROR
35,ERROR,multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,a77fd83f-bc94-4da8-9049-2f241727b854genid324065,NaN,ERROR
36,ERROR,multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,a77fd83f-bc94-4da8-9049-2f241727b854genid350451,NaN,ERROR
37,ERROR,multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,a77fd83f-bc94-4da8-9049-2f241727b854genid350455,NaN,ERROR


# Other checks analyses

In [22]:
report = '/Users/matentzn/ws/mondo/src/ontology/reports/single-child-warning.tsv'
df_scw = pd.read_csv(report,sep='\t') 
df_scw 
# Single child classes should be tagged. We should review them one by one.

,?cls,?total
0,<http://purl.obolibrary.org/obo/MONDO_0016094>,1
1,<http://purl.obolibrary.org/obo/MONDO_0021396>,1
2,<http://purl.obolibrary.org/obo/MONDO_0016093>,1
3,<http://purl.obolibrary.org/obo/MONDO_0008882>,1
4,<http://purl.obolibrary.org/obo/MONDO_0006224>,1
...,...,...
1558,<http://purl.obolibrary.org/obo/MONDO_0000461>,1
1559,<http://purl.obolibrary.org/obo/MONDO_0021540>,1
1560,<http://purl.obolibrary.org/obo/MONDO_0005846>,1
1561,<http://purl.obolibrary.org/obo/MONDO_0004648>,1


report = '/Users/matentzn/ws/mondo/src/ontology/reports/two-label-violation.tsv'
df_tlw = pd.read_csv(report,sep='\t') 
df_tlw

In [23]:
report = '/Users/matentzn/ws/mondo/src/ontology/reports/no-subclass-between-genetic-disease-warning.tsv'
df_nsbgd = pd.read_csv(report,sep='\t') 
df_nsbgd
## These cases should be tagged as well. But lets review them bit by bit


,?d1,?d2
0,<http://purl.obolibrary.org/obo/MONDO_0010735>,<http://purl.obolibrary.org/obo/MONDO_0009669>
1,<http://purl.obolibrary.org/obo/MONDO_0011211>,<http://purl.obolibrary.org/obo/MONDO_0009943>
2,<http://purl.obolibrary.org/obo/MONDO_0013966>,<http://purl.obolibrary.org/obo/MONDO_0008648>
3,<http://purl.obolibrary.org/obo/MONDO_0044304>,<http://purl.obolibrary.org/obo/MONDO_0009861>
4,<http://purl.obolibrary.org/obo/MONDO_0014191>,<http://purl.obolibrary.org/obo/MONDO_0008648>
...,...,...
62,<http://purl.obolibrary.org/obo/MONDO_0007794>,<http://purl.obolibrary.org/obo/MONDO_0014461>
63,<http://purl.obolibrary.org/obo/MONDO_0014722>,<http://purl.obolibrary.org/obo/MONDO_0010248>
64,<http://purl.obolibrary.org/obo/MONDO_0020746>,<http://purl.obolibrary.org/obo/MONDO_0009926>
65,<http://purl.obolibrary.org/obo/MONDO_0012762>,<http://purl.obolibrary.org/obo/MONDO_0008648>


In [25]:
report = '/Users/matentzn/ws/mondo/src/ontology/reports/related-exact-synonym-warning.tsv'
df_res = pd.read_csv(report,sep='\t') 
df_res
# migrate provenance over after duplicate_exact_synonym check..
# Make tsv we should be strict
# label in one term should not be equal to exact synonym
# unique name should be unique

,?cls,?label,?related
0,<http://purl.obolibrary.org/obo/MONDO_0014756>,"tremor, hereditary essential, 5",ETM5
1,<http://purl.obolibrary.org/obo/MONDO_0006999>,tooth disease,disorder of calcareous tooth
2,<http://purl.obolibrary.org/obo/MONDO_0009424>,Bartter disease type 2,hyperprostaglandin E syndrome 2
3,<http://purl.obolibrary.org/obo/MONDO_0017686>,inborn aminoacylase deficiency,rare inborn error of aminoacylase activity
4,<http://purl.obolibrary.org/obo/MONDO_0020680>,acute bronchiolitis,acute bronchiolitis
...,...,...,...
195,<http://purl.obolibrary.org/obo/MONDO_0007068>,adenylosuccinate lyase deficiency,rare inborn error of (S)-2-(5-amino-1-(5-phosp...
196,<http://purl.obolibrary.org/obo/MONDO_0009626>,pseudo-TORCH syndrome,band-like calcification with simplified gyrati...
197,<http://purl.obolibrary.org/obo/MONDO_0003646>,rectum neuroendocrine neoplasm,rectum neuroendocrine tumor
198,<http://purl.obolibrary.org/obo/MONDO_0007435>,dentatorubral-pallidoluysian atrophy,DRPLA


In [28]:
report = '/Users/matentzn/ws/mondo/src/ontology/reports/excluded-subsumption-is-inferred.tsv'
df_esi = pd.read_csv(report,sep='\t') 
df_esi

,?cls,?label,?excluded,?excluded_label
0,<http://purl.obolibrary.org/obo/MONDO_0043174>,Pfeiffer Tietze Welte syndrome,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
1,<http://purl.obolibrary.org/obo/MONDO_0002722>,olfactory nerve neoplasm,<http://purl.obolibrary.org/obo/MONDO_0002722>,olfactory nerve neoplasm
2,<http://purl.obolibrary.org/obo/MONDO_0023059>,Elliott ludman Teebi syndrome,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
3,<http://purl.obolibrary.org/obo/MONDO_0024524>,dyschromatosis universalis hereditaria 1,<http://purl.obolibrary.org/obo/MONDO_0006600>,pigmentation disease
4,<http://purl.obolibrary.org/obo/MONDO_0018695>,avian influenza,<http://purl.obolibrary.org/obo/MONDO_0005087>,respiratory system disease
...,...,...,...,...
97,<http://purl.obolibrary.org/obo/MONDO_0044322>,intellectual developmental disorder with neuro...,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
98,<http://purl.obolibrary.org/obo/MONDO_0001650>,acute cystitis (disease),<http://purl.obolibrary.org/obo/MONDO_0006032>,cystitis
99,<http://purl.obolibrary.org/obo/MONDO_0001096>,mediastinum ganglioneuroblastoma,<http://purl.obolibrary.org/obo/MONDO_0005843>,mediastinal cancer
100,<http://purl.obolibrary.org/obo/MONDO_0043071>,Zazam Sheriff Phillips syndrome,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
